In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import scipy.stats
import scipy.optimize
import tqdm.autonotebook as tqdman

In [ ]:
OUTPUT_ENSEMBLE_SIZE = 1001
NINETY_TO_ONESIGMA = scipy.stats.norm.ppf(0.95)

In [ ]:
valid_temp = np.loadtxt('../data/ar6_ensemble_batches_rcp/accepted_rmse_temp.csv', dtype=int)

In [ ]:
input_ensemble_size = len(valid_temp)
input_ensemble_size

In [ ]:
assert input_ensemble_size > OUTPUT_ENSEMBLE_SIZE

In [ ]:
temp_in = np.load('../data/ar6_ensemble_batches_rcp/temperature_1850-2100.npy')
ohc_in = np.load('../data/ar6_ensemble_batches_rcp/ohc_2018_minus_1971.npy')
fari_in = np.load('../data/ar6_ensemble_batches_rcp/fari_2005-2014_mean.npy')
faci_in = np.load('../data/ar6_ensemble_batches_rcp/faci_2005-2014_mean.npy')
co2_in = np.load('../data/ar6_ensemble_batches_rcp/co2_2014.npy')
ecs_in = np.load('../data/ar6_ensemble_batches_rcp/ecs.npy')
tcr_in = np.load('../data/ar6_ensemble_batches_rcp/tcr.npy')

In [ ]:
def opt(x, q05_desired, q50_desired, q95_desired):
    "x is (a, loc, scale) in that order."
    q05, q50, q95 = scipy.stats.skewnorm.ppf((0.05, 0.50, 0.95), x[0], loc=x[1], scale=x[2])
    #print(q05, q50, q95, x)
    return (q05-q05_desired, q50-q50_desired, q95-q95_desired)
    
scipy.optimize.root(opt, (2.0, 2.6, 1.2), (1.24, 1.81, 2.59), options={'maxfev': 5000})

In [ ]:
samples = {}
samples['ECS'] = scipy.stats.skewnorm.rvs(8.82185594, loc=1.95059779, scale=1.55584604, size=10**5, random_state=91603)
samples['TCR'] = scipy.stats.norm.rvs(loc=1.8, scale=0.6/NINETY_TO_ONESIGMA, size=10**5, random_state=18196)
samples['OHC'] = scipy.stats.norm.rvs(loc=434.3730268913012, scale=73.70562981598447, size=10**5, random_state=43178)
samples['temperature 1995-2014'] = scipy.stats.skewnorm.rvs(-1.65506091, loc=0.92708099, scale=0.12096636, size=10**5, random_state=19387)
#samples['temperature 2081-2100'] = scipy.stats.skewnorm.rvs(2.20496701, loc=1.4124379, scale=0.60080822, size=10**5, random_state=91831)
samples['ERFari'] = scipy.stats.norm.rvs(loc=-0.3, scale=0.3/NINETY_TO_ONESIGMA, size=10**5, random_state=70173)
samples['ERFaci'] = scipy.stats.norm.rvs(loc=-1.0, scale=0.7/NINETY_TO_ONESIGMA, size=10**5, random_state=91123)
samples['CO2 concentration'] = scipy.stats.norm.rvs(loc=397.5469792683919, scale=0.36, size=10**5, random_state=81693)

In [ ]:
pl.hist(samples['ECS'], bins=100, alpha=0.5, label='Sampled', density=True)
pl.hist(ecs_in[valid_temp], bins=100, alpha=0.5, label='Naive constraints', density=True)
pl.legend()
print('target distribution:', np.percentile(samples['ECS'], (5, 16, 50, 84, 95)))
print('naive constraints:', np.percentile(ecs_in[valid_temp], (5, 16, 50, 84, 95)))

In [ ]:
ar_distributions = {}
#for constraint in ['ECS', 'TCR', 'OHC', 'temperature 1995-2014', 'temperature 2081-2100', 'ERFari', 'ERFaci', 'CO2 concentration']:
for constraint in ['ECS', 'TCR', 'OHC', 'temperature 1995-2014', 'ERFari', 'ERFaci', 'CO2 concentration']:
    ar_distributions[constraint] = {}
    ar_distributions[constraint]['bins'] = np.histogram(samples[constraint], bins=100, density=True)[1]
    ar_distributions[constraint]['values'] = samples[constraint]

In [ ]:
accepted = pd.DataFrame(
    {
        'ECS': ecs_in[valid_temp],
        'TCR': tcr_in[valid_temp],
        'OHC': ohc_in[valid_temp]/1e21,
        'temperature 1995-2014': temp_in[145:165,valid_temp].mean(axis=0) - temp_in[:51,valid_temp].mean(axis=0),
#        'temperature 2081-2100': temp_in[231:251,valid_temp].mean(axis=0) - temp_in[145:165,valid_temp].mean(axis=0),
        'ERFari': fari_in[valid_temp],
        'ERFaci': faci_in[valid_temp],
        'CO2 concentration': co2_in[valid_temp]
    },
    index=valid_temp
)

In [ ]:
accepted

In [ ]:
def calculate_sample_weights(distributions, samples, niterations=50):
    #weights = np.ones(samples[list(accepted.keys())[0]].shape[0])
    weights = np.ones(samples.shape[0])
    gofs = []
    gofs_full = []

    unique_codes = list(distributions.keys())#[::-1]

    for k in tqdman.tqdm(range(niterations), desc="Iterations", leave=False):
        gofs.append([])
        if k == (niterations - 1):
            weights_second_last_iteration = weights.copy()
            weights_to_average = []
            
        for j, unique_code in enumerate(unique_codes):
            unique_code_weights, our_values_bin_idx = get_unique_code_weights(
                unique_code, distributions, samples, weights, j, k
            )
            if k == (niterations - 1):
                weights_to_average.append(unique_code_weights[our_values_bin_idx])
        
            weights *= unique_code_weights[our_values_bin_idx]
            
            gof = ((unique_code_weights[1:-1] - 1) ** 2).sum()
            gofs[-1].append(gof)
               
            gofs_full.append([unique_code])
            for unique_code_check in unique_codes:
                unique_code_check_weights, _ = get_unique_code_weights(
                    unique_code_check, distributions, samples, weights, 1, 1
                )
                gof = ((unique_code_check_weights[1:-1] - 1) ** 2).sum()
                gofs_full[-1].append(gof)

    weights_stacked = np.vstack(weights_to_average).mean(axis=0)
    weights_final = weights_stacked * weights_second_last_iteration

    gofs_full.append(["Final iteration"])
    for unique_code_check in unique_codes:
        unique_code_check_weights, _ = get_unique_code_weights(
            unique_code_check, distributions, samples, weights_final, 1, 1
        )
        gof = ((unique_code_check_weights[1:-1] - 1) ** 2).sum()
        gofs_full[-1].append(gof)

    return (
        weights_final, 
        pd.DataFrame(np.array(gofs), columns=unique_codes), 
        pd.DataFrame(np.array(gofs_full), columns=["Target marginal"] + unique_codes)
    )


In [ ]:
def get_unique_code_weights(unique_code, distributions, samples, weights, j, k):
    bin_edges = distributions[unique_code]["bins"]
    our_values = samples[unique_code].copy()

    our_values_bin_counts, bin_edges_np = np.histogram(
        our_values, bins=bin_edges
    )
    np.testing.assert_allclose(bin_edges, bin_edges_np)
    assessed_ranges_bin_counts, _ = np.histogram(
        distributions[unique_code]["values"], bins=bin_edges
    )
    
    our_values_bin_idx = np.digitize(our_values, bins=bin_edges)

    existing_weighted_bin_counts = np.nan * np.zeros(
        our_values_bin_counts.shape[0]
    )
    for i in range(existing_weighted_bin_counts.shape[0]):
        existing_weighted_bin_counts[i] = weights[
            (our_values_bin_idx == i + 1)
        ].sum()

    if np.equal(j, 0) and np.equal(k, 0):
        np.testing.assert_equal(
            existing_weighted_bin_counts.sum(), our_values_bin_counts.sum()
        )

    unique_code_weights = np.nan * np.zeros(bin_edges.shape[0] + 1)

    # existing_weighted_bin_counts[0] refers to samples outside the
    # assessed range's lower bound. Accordingly, if `our_values` was
    # digitized into a bin idx of zero, it should get a weight of zero.
    unique_code_weights[0] = 0
    # Similarly, if `our_values` was digitized into a bin idx greater
    # than the number of bins then it was outside the assessed range
    # so get a weight of zero.
    unique_code_weights[-1] = 0
    
    for i in range(1, our_values_bin_counts.shape[0] + 1):
        # the histogram idx is one less because digitize gives values in the
        # range bin_edges[0] <= x < bin_edges[1] a digitized index of 1
        histogram_idx = i - 1
        if np.equal(assessed_ranges_bin_counts[histogram_idx], 0):
            unique_code_weights[i] = 0
        elif np.equal(existing_weighted_bin_counts[histogram_idx], 0):
            # other variables force this box to be zero so just fill it with
            # one
            unique_code_weights[i] = 1
        else:
            unique_code_weights[i] = (
                assessed_ranges_bin_counts[histogram_idx]
                / existing_weighted_bin_counts[histogram_idx]
            )
            
    return unique_code_weights, our_values_bin_idx


In [ ]:
weights, gofs, gofs_full = calculate_sample_weights(
    ar_distributions, accepted, niterations=30
)

In [ ]:
gofs_full

In [ ]:
weights

In [ ]:
gofs

In [ ]:
effective_samples = int(np.floor(np.sum(np.minimum(weights, 1))))
effective_samples

In [ ]:
draws = []

drawn_samples = accepted.sample(n=OUTPUT_ENSEMBLE_SIZE, replace=False, weights=weights, random_state=10099)
draws.append((drawn_samples))

#for i in tqdman.tqdm(range(10)):
#    drawn_samples = accepted.sample(n=OUTPUT_ENSEMBLE_SIZE, replace=False, weights=weights, random_state=)
    #     drawn_samples = samples_df.sample(n=10, replace=False, weights=weights)

    #summary_table = get_summary_table(drawn_samples)

    #score, coloured_df = utils.plotting.colour_df(summary_table, transpose=True)

#    draws.append((drawn_samples))

#sorted([v[1] for v in draws])[:10]

In [ ]:
draws

In [ ]:
pl.hist(draws[0]['ECS'])
np.percentile(draws[0]['ECS'], (5, 50, 95))

In [ ]:
pl.hist(draws[0]['TCR'])
np.percentile(draws[0]['TCR'], (5, 50, 95))

In [ ]:
pl.scatter(draws[0]['TCR'], draws[0]['ECS'])

In [ ]:
pl.hist(draws[0]['CO2 concentration'])
np.percentile(draws[0]['CO2 concentration'], (5, 50, 95))

In [ ]:
pl.hist(draws[0]['temperature 1995-2014'])
np.percentile(draws[0]['temperature 1995-2014'], (5, 50, 95))

In [ ]:
pl.hist(draws[0]['ERFari'])
np.percentile(draws[0]['ERFari'], (5, 50, 95))

In [ ]:
pl.hist(draws[0]['ERFaci'])
np.percentile(draws[0]['ERFaci'], (5, 50, 95))

In [ ]:
pl.hist(draws[0]['ERFaci']+draws[0]['ERFari'])
np.percentile(draws[0]['ERFaci']+draws[0]['ERFari'], (5, 50, 95))

In [ ]:
pl.hist(draws[0]['OHC'])
np.percentile(draws[0]['OHC'], (5, 50, 95))

In [ ]:
pl.scatter(draws[0]['TCR'], draws[0]['ERFaci']+draws[0]['ERFari'])

In [ ]:
draws[0].index

In [ ]:
df_gmst = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = df_gmst['gmst'].values

In [ ]:
fig, ax = pl.subplots()
ax.fill_between(
    np.arange(1850.5, 2101), 
    np.min(temp_in[:,draws[0].index]-temp_in[:51,draws[0].index].mean(axis=0), axis=1), 
    np.max(temp_in[:,draws[0].index]-temp_in[:51,draws[0].index].mean(axis=0), axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1850.5, 2101), 
    np.percentile(temp_in[:,draws[0].index]-temp_in[:51,draws[0].index].mean(axis=0), 5, axis=1), 
    np.percentile(temp_in[:,draws[0].index]-temp_in[:51,draws[0].index].mean(axis=0), 95, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1850.5, 2101), 
    np.percentile(temp_in[:,draws[0].index]-temp_in[:51,draws[0].index].mean(axis=0), 16, axis=1), 
    np.percentile(temp_in[:,draws[0].index]-temp_in[:51,draws[0].index].mean(axis=0), 84, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.plot(
    np.arange(1850.5, 2101), 
    np.median(temp_in[:,draws[0].index]-temp_in[:51,draws[0].index].mean(axis=0), axis=1), 
    color='#000000',
)

ax.plot(np.arange(1850.5, 2021), gmst, color='b')

ax.set_xlim(1850,2100)
ax.set_ylim(-0.6, 3.5)
ax.axhline(0, color='k', ls=":", lw=0.5)
pl.title('Temperature anomaly - constrained')

In [ ]:
np.savetxt('../data/ar6_ensemble_batches_rcp/final.csv', sorted(draws[0].index))

In [ ]:
np.median(temp_in[-1,draws[0].index]-temp_in[:51,draws[0].index].mean(axis=0))